In [ ]:
import os
import numpy as np
from scipy.signal import butter, filtfilt
import networkx as nx
from scipy.ndimage import zoom
import matplotlib.pyplot as plt


def load_and_validate_data(file_path):
    """
    Load data from a file and validate its presence.
    Return None if data is invalid or shapes are inconsistent.
    """
    try:
        loaded_data = np.load(file_path)
        
        # Extract data
        ppg_f = loaded_data.get('ppg_f')
        ecg_f = loaded_data.get('ecg_f')
        seg_dbp = loaded_data.get('seg_dbp')
        seg_sbp = loaded_data.get('seg_sbp')
        
        # Check if all required data is present and non-null
        if ppg_f is None or ecg_f is None or seg_dbp is None or seg_sbp is None:
            return None
        
        # Check for empty data
        if ppg_f.size == 0 or ecg_f.size == 0 or seg_dbp.size == 0 or seg_sbp.size == 0:
            return None

        return ppg_f, ecg_f, seg_dbp, seg_sbp
    
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

In [ ]:
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    """
    Apply a bandpass filter to the data.
    """
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

def preprocess_signal(signal, fs):
    """
    Preprocess the signal by bandpass filtering and normalization.
    """
    filtered_signal = bandpass_filter(signal, 0.5, 40, fs)
    normalized_signal = (filtered_signal - np.mean(filtered_signal)) / np.std(filtered_signal)
    return normalized_signal

In [ ]:
def create_visibility_graph(ppg_signal):
    n = len(ppg_signal)
    G = nx.Graph()
    G.add_nodes_from(range(n))

    for i in range(n):
        for j in range(i + 1, n):
            visible = True
            for k in range(i + 1, j):
                if ppg_signal[k] >= ppg_signal[i] + (ppg_signal[j] - ppg_signal[i]) * (k - i) / (j - i):
                    visible = False
                    break
            if visible:
                G.add_edge(i, j)
                
    return G

def graph_to_adjacency_matrix_image(G, size):
    adj_matrix = nx.to_numpy_array(G)
    # Resize adjacency matrix to a fixed size
    adj_matrix_resized = zoom(adj_matrix, (size / adj_matrix.shape[0], size / adj_matrix.shape[1]), order=0)
    return adj_matrix_resized

def generate_vg_image(ppg_signal, size):
    G = create_visibility_graph(ppg_signal)
    vg_image = graph_to_adjacency_matrix_image(G, size)
    return vg_image

def plot_vg_image(vg_image):
    plt.figure(figsize=(6, 6))
    plt.imshow(vg_image, cmap='gray', interpolation='nearest')
    plt.title('Visibility Graph Image')
    plt.colorbar()
    plt.show()

In [ ]:
def combine_data_from_folder(folder_path, vg_image_size, batch_size=100):
    combined_ppg = []
    combined_ecg = []
    combined_seg_dbp = []
    combined_seg_sbp = []
    combined_vg_images = []

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        if not file_path.endswith('.npz'):
            continue
        
        data = load_and_validate_data(file_path)
        
        if data is None:
            print(f"Skipping invalid file: {file_path}")
            continue
        
        ppg_f, ecg_f, seg_dbp, seg_sbp = data
        
        if ppg_f.ndim == 3:
            for i in range(ppg_f.shape[0]):
                preprocessed_ppg = preprocess_signal(ppg_f[i], 1000)
                combined_ppg.append(preprocessed_ppg)
                vg_image = generate_vg_image(preprocessed_ppg, vg_image_size)
                combined_vg_images.append(vg_image)
        else:
            print(f"Skipping PPG data with unexpected shape: {ppg_f.shape}")
        
        if ecg_f.ndim == 3:
            for i in range(ecg_f.shape[0]):
                preprocessed_ecg = preprocess_signal(ecg_f[i], 1000)
                combined_ecg.append(preprocessed_ecg)
        else:
            print(f"Skipping ECG data with unexpected shape: {ecg_f.shape}")
        
        if seg_dbp.ndim == 2:
            combined_seg_dbp.append(seg_dbp)
        else:
            print(f"Skipping SegDBP data with unexpected shape: {seg_dbp.shape}")
        
        if seg_sbp.ndim == 2:
            combined_seg_sbp.append(seg_sbp)
        else:
            print(f"Skipping SegSBP data with unexpected shape: {seg_sbp.shape}")
        
        if len(combined_ppg) >= batch_size:
            combined_ppg = np.stack(combined_ppg, axis=0)
            combined_ecg = np.stack(combined_ecg, axis=0)
            combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
            combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
            combined_vg_images = np.stack(combined_vg_images, axis=0)
            
            yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp, combined_vg_images
            
            combined_ppg = []
            combined_ecg = []
            combined_seg_dbp = []
            combined_seg_sbp = []
            combined_vg_images = []

    if combined_ppg:
        combined_ppg = np.stack(combined_ppg, axis=0)
    else:
        combined_ppg = np.array([])
        
    if combined_ecg:
        combined_ecg = np.stack(combined_ecg, axis=0)
    else:
        combined_ecg = np.array([])
        
    if combined_seg_dbp:
        combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
    else:
        combined_seg_dbp = np.array([])
        
    if combined_seg_sbp:
        combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
    else:
        combined_seg_sbp = np.array([])
        
    if combined_vg_images:
        combined_vg_images = np.stack(combined_vg_images, axis=0)
    else:
        combined_vg_images = np.array([])

    yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp, combined_vg_images

# Define folder path containing .npz files
folder_path = r'C:\Users\nihal\Desktop\xyz'
vg_image_size = 224  # Define the size for VG images

# Combine data from all valid files
combined_data_generator = combine_data_from_folder(folder_path, vg_image_size)

# Plot VG images
for batch in combined_data_generator:
    combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp, combined_vg_images = batch
    for i in range(min(5, len(combined_vg_images))):  # Plot up to 5 images
        plot_vg_image(combined_vg_images[i])
    break  # Only process the first batch for plotting

# COMPLETE CODE FOR EXTRACTING DATA FROM FOLDERS AND GENERATING VG IMAGES

In [1]:
import os
import numpy as np
from scipy.signal import butter, filtfilt
import networkx as nx
from scipy.ndimage import zoom
import matplotlib.pyplot as plt

def load_and_validate_data(file_path):
    """
    Load data from a file and validate its presence.
    Return None if data is invalid or shapes are inconsistent.
    """
    try:
        loaded_data = np.load(file_path)
        
        # Extract data
        ppg_f = loaded_data.get('ppg_f')
        ecg_f = loaded_data.get('ecg_f')
        seg_dbp = loaded_data.get('seg_dbp')
        seg_sbp = loaded_data.get('seg_sbp')
        
        # Check if all required data is present and non-null
        if ppg_f is None or ecg_f is None or seg_dbp is None or seg_sbp is None:
            return None
        
        # Check for empty data
        if ppg_f.size == 0 or ecg_f.size == 0 or seg_dbp.size == 0 or seg_sbp.size == 0:
            return None

        return ppg_f, ecg_f, seg_dbp, seg_sbp
    
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    """
    Apply a bandpass filter to the data.
    """
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

def preprocess_signal(signal, fs):
    """
    Preprocess the signal by bandpass filtering and normalization.
    """
    filtered_signal = bandpass_filter(signal, 0.5, 40, fs)
    normalized_signal = (filtered_signal - np.mean(filtered_signal)) / np.std(filtered_signal)
    return normalized_signal

def create_visibility_graph(ppg_signal):
    n = len(ppg_signal)
    G = nx.Graph()
    G.add_nodes_from(range(n))

    for i in range(n):
        for j in range(i + 1, n):
            visible = True
            for k in range(i + 1, j):
                if ppg_signal[k] >= ppg_signal[i] + (ppg_signal[j] - ppg_signal[i]) * (k - i) / (j - i):
                    visible = False
                    break
            if visible:
                G.add_edge(i, j)
                
    return G

def graph_to_adjacency_matrix_image(G, size):
    adj_matrix = nx.to_numpy_array(G)
    adj_matrix_resized = zoom(adj_matrix, (size / adj_matrix.shape[0], size / adj_matrix.shape[1]), order=0)
    return adj_matrix_resized

def graph_to_flattened_adjacency_matrix(G, size):
    adj_matrix = nx.to_numpy_array(G)
    adj_matrix_resized = zoom(adj_matrix, (size / adj_matrix.shape[0], size / adj_matrix.shape[1]), order=0)
    flattened_adj = adj_matrix_resized.flatten()
    return flattened_adj[:size * size]  # Ensure it's the correct size

def generate_vg_image(ppg_signal, size):
    G = create_visibility_graph(ppg_signal)
    vg_image = graph_to_adjacency_matrix_image(G, size)
    return vg_image

def plot_vg_images(vg_images):
    plt.figure(figsize=(10, 10))
    num_images = len(vg_images)
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(vg_images[i], cmap='gray')
        plt.axis('off')
    plt.show()
    
def combine_data_from_folder(folder_path , batch_size=100):
    """
    Combine and preprocess data from all valid files in the folder and generate VG images.
    """
    combined_ppg = []
    combined_ecg = []
    combined_seg_dbp = []
    combined_seg_sbp = []

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        if not file_path.endswith('.npz'):
            continue
        
        data = load_and_validate_data(file_path)
        
        if data is None:
            print(f"Skipping invalid file: {file_path}")
            continue
        
        ppg_f, ecg_f, seg_dbp, seg_sbp = data
        
        if ppg_f.ndim == 3:
            for i in range(ppg_f.shape[0]):
                preprocessed_ppg = preprocess_signal(ppg_f[i], 1000)
                combined_ppg.append(preprocessed_ppg)
        else:
            print(f"Skipping PPG data with unexpected shape: {ppg_f.shape}")
        
        if ecg_f.ndim == 3:
            for i in range(ecg_f.shape[0]):
                preprocessed_ecg = preprocess_signal(ecg_f[i], 1000)
                combined_ecg.append(preprocessed_ecg)
        else:
            print(f"Skipping ECG data with unexpected shape: {ecg_f.shape}")
        
        if seg_dbp.ndim == 2:
            combined_seg_dbp.append(seg_dbp)
        else:
            print(f"Skipping SegDBP data with unexpected shape: {seg_dbp.shape}")
        
        if seg_sbp.ndim == 2:
            combined_seg_sbp.append(seg_sbp)
        else:
            print(f"Skipping SegSBP data with unexpected shape: {seg_sbp.shape}")
        
        if len(combined_ppg) >= batch_size:
            combined_ppg = np.stack(combined_ppg, axis=0)
            combined_ecg = np.stack(combined_ecg, axis=0)
            combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
            combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
            
            yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
            
            combined_ppg = []
            combined_ecg = []
            combined_seg_dbp = []
            combined_seg_sbp = []
            

    if combined_ppg:
        combined_ppg = np.stack(combined_ppg, axis=0)
    else:
        combined_ppg = np.array([])
        
    if combined_ecg:
        combined_ecg = np.stack(combined_ecg, axis=0)
    else:
        combined_ecg = np.array([])
        
    if combined_seg_dbp:
        combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
    else:
        combined_seg_dbp = np.array([])
        
    if combined_seg_sbp:
        combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
    else:
        combined_seg_sbp = np.array([])

    yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp

# Function to apply PCA on a batch of VG images
def apply_pca_to_images(vg_images, n_components):
    # Flatten all images to create a batch
    flattened_images = np.array([image.flatten() for image in vg_images])

    # Check if we have enough samples and features to apply PCA
    n_samples, n_features = flattened_images.shape
    if n_samples > 1 and n_features > n_components:
        pca = PCA(n_components=n_components)
        reduced_images = pca.fit_transform(flattened_images)
        return reduced_images
    else:
        # If not enough samples or features, return the original flattened images
        return flattened_images
    
def invert_images(vg_images):
    # Invert pixel values (assuming images are in [0, 255] range)
    inverted_images = [255 - image for image in vg_images]
    return inverted_images

# Define folder path containing .npz files
folder_path = r'C:\Users\nihal\Desktop\NIHAL_IMP_DOCS\Internship_PPG\processed_npz_files_VitalDB_Final'
#folder_path = r'C:\Users\nihal\Desktop\xyz'
vg_image_size = 224  # Define the size for VG images

# Combine data from all valid files
combined_data_generator = combine_data_from_folder(folder_path, vg_image_size)

# THE BELOW CODE IS FOR VISUALIZING TSNE FOR SINGLE BATCH

In [2]:
import numpy as np
import os

# Create a directory to save VG images if it doesn't exist
output_dir = 'vg_images'
os.makedirs(output_dir, exist_ok=True)

for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch) in enumerate(combined_data_generator):
    print(f"Processing Batch {batch_idx + 1}...")

    vg_images = []
    inverted_vg_images = []
    
    for i in range(len(combined_ppg_batch)):
        print(f"Generating VG image for PPG signal {i + 1}/{len(combined_ppg_batch)}")
        
        ppg_signal = combined_ppg_batch[i].flatten()
        
        # Generate the VG image for the PPG signal
        vg_image = generate_vg_image(ppg_signal, vg_image_size)
        vg_images.append(vg_image.flatten())
        
        # Generate the inverted VG image
        inverted_vg_image = invert_images([vg_image])[0]
        inverted_vg_images.append(inverted_vg_image.flatten())

    # Save VG images to disk
    np.save(os.path.join(output_dir, f'vg_images_batch_{batch_idx + 1}.npy'), vg_images)
    np.save(os.path.join(output_dir, f'inverted_vg_images_batch_{batch_idx + 1}.npy'), inverted_vg_images)

    print(f"Batch {batch_idx + 1} processing complete.")

    # Optionally break after the first batch
    


Processing Batch 1...
Generating VG image for PPG signal 1/916
Generating VG image for PPG signal 2/916
Generating VG image for PPG signal 3/916
Generating VG image for PPG signal 4/916
Generating VG image for PPG signal 5/916
Generating VG image for PPG signal 6/916
Generating VG image for PPG signal 7/916
Generating VG image for PPG signal 8/916
Generating VG image for PPG signal 9/916
Generating VG image for PPG signal 10/916
Generating VG image for PPG signal 11/916
Generating VG image for PPG signal 12/916
Generating VG image for PPG signal 13/916
Generating VG image for PPG signal 14/916
Generating VG image for PPG signal 15/916
Generating VG image for PPG signal 16/916
Generating VG image for PPG signal 17/916
Generating VG image for PPG signal 18/916
Generating VG image for PPG signal 19/916
Generating VG image for PPG signal 20/916
Generating VG image for PPG signal 21/916
Generating VG image for PPG signal 22/916
Generating VG image for PPG signal 23/916
Generating VG image f

Generating VG image for PPG signal 194/916
Generating VG image for PPG signal 195/916
Generating VG image for PPG signal 196/916
Generating VG image for PPG signal 197/916
Generating VG image for PPG signal 198/916
Generating VG image for PPG signal 199/916
Generating VG image for PPG signal 200/916
Generating VG image for PPG signal 201/916
Generating VG image for PPG signal 202/916
Generating VG image for PPG signal 203/916
Generating VG image for PPG signal 204/916
Generating VG image for PPG signal 205/916
Generating VG image for PPG signal 206/916
Generating VG image for PPG signal 207/916
Generating VG image for PPG signal 208/916
Generating VG image for PPG signal 209/916
Generating VG image for PPG signal 210/916
Generating VG image for PPG signal 211/916
Generating VG image for PPG signal 212/916
Generating VG image for PPG signal 213/916
Generating VG image for PPG signal 214/916
Generating VG image for PPG signal 215/916
Generating VG image for PPG signal 216/916
Generating 

Generating VG image for PPG signal 385/916
Generating VG image for PPG signal 386/916
Generating VG image for PPG signal 387/916
Generating VG image for PPG signal 388/916
Generating VG image for PPG signal 389/916
Generating VG image for PPG signal 390/916
Generating VG image for PPG signal 391/916
Generating VG image for PPG signal 392/916
Generating VG image for PPG signal 393/916
Generating VG image for PPG signal 394/916
Generating VG image for PPG signal 395/916
Generating VG image for PPG signal 396/916
Generating VG image for PPG signal 397/916
Generating VG image for PPG signal 398/916
Generating VG image for PPG signal 399/916
Generating VG image for PPG signal 400/916
Generating VG image for PPG signal 401/916
Generating VG image for PPG signal 402/916
Generating VG image for PPG signal 403/916
Generating VG image for PPG signal 404/916
Generating VG image for PPG signal 405/916
Generating VG image for PPG signal 406/916
Generating VG image for PPG signal 407/916
Generating 

Generating VG image for PPG signal 576/916
Generating VG image for PPG signal 577/916
Generating VG image for PPG signal 578/916
Generating VG image for PPG signal 579/916
Generating VG image for PPG signal 580/916
Generating VG image for PPG signal 581/916
Generating VG image for PPG signal 582/916
Generating VG image for PPG signal 583/916
Generating VG image for PPG signal 584/916
Generating VG image for PPG signal 585/916
Generating VG image for PPG signal 586/916
Generating VG image for PPG signal 587/916
Generating VG image for PPG signal 588/916
Generating VG image for PPG signal 589/916
Generating VG image for PPG signal 590/916
Generating VG image for PPG signal 591/916
Generating VG image for PPG signal 592/916
Generating VG image for PPG signal 593/916
Generating VG image for PPG signal 594/916
Generating VG image for PPG signal 595/916
Generating VG image for PPG signal 596/916
Generating VG image for PPG signal 597/916
Generating VG image for PPG signal 598/916
Generating 

Generating VG image for PPG signal 767/916
Generating VG image for PPG signal 768/916
Generating VG image for PPG signal 769/916
Generating VG image for PPG signal 770/916
Generating VG image for PPG signal 771/916
Generating VG image for PPG signal 772/916
Generating VG image for PPG signal 773/916
Generating VG image for PPG signal 774/916
Generating VG image for PPG signal 775/916
Generating VG image for PPG signal 776/916
Generating VG image for PPG signal 777/916
Generating VG image for PPG signal 778/916
Generating VG image for PPG signal 779/916
Generating VG image for PPG signal 780/916
Generating VG image for PPG signal 781/916
Generating VG image for PPG signal 782/916
Generating VG image for PPG signal 783/916
Generating VG image for PPG signal 784/916
Generating VG image for PPG signal 785/916
Generating VG image for PPG signal 786/916
Generating VG image for PPG signal 787/916
Generating VG image for PPG signal 788/916
Generating VG image for PPG signal 789/916
Generating 

...

Generating VG image for PPG signal 1057/1538
Generating VG image for PPG signal 1058/1538
Generating VG image for PPG signal 1059/1538
Generating VG image for PPG signal 1060/1538
Generating VG image for PPG signal 1061/1538
Generating VG image for PPG signal 1062/1538
Generating VG image for PPG signal 1063/1538
Generating VG image for PPG signal 1064/1538
Generating VG image for PPG signal 1065/1538
Generating VG image for PPG signal 1066/1538
Generating VG image for PPG signal 1067/1538
Generating VG image for PPG signal 1068/1538
Generating VG image for PPG signal 1069/1538
Generating VG image for PPG signal 1070/1538
Generating VG image for PPG signal 1071/1538
Generating VG image for PPG signal 1072/1538
Generating VG image for PPG signal 1073/1538
Generating VG image for PPG signal 1074/1538
Generating VG image for PPG signal 1075/1538
Generating VG image for PPG signal 1076/1538
Generating VG image for PPG signal 1077/1538
Generating VG image for PPG signal 1078/1538
Generating

Generating VG image for PPG signal 1240/1538
Generating VG image for PPG signal 1241/1538
Generating VG image for PPG signal 1242/1538
Generating VG image for PPG signal 1243/1538
Generating VG image for PPG signal 1244/1538
Generating VG image for PPG signal 1245/1538
Generating VG image for PPG signal 1246/1538
Generating VG image for PPG signal 1247/1538
Generating VG image for PPG signal 1248/1538
Generating VG image for PPG signal 1249/1538
Generating VG image for PPG signal 1250/1538
Generating VG image for PPG signal 1251/1538
Generating VG image for PPG signal 1252/1538
Generating VG image for PPG signal 1253/1538
Generating VG image for PPG signal 1254/1538
Generating VG image for PPG signal 1255/1538
Generating VG image for PPG signal 1256/1538
Generating VG image for PPG signal 1257/1538
Generating VG image for PPG signal 1258/1538
Generating VG image for PPG signal 1259/1538
Generating VG image for PPG signal 1260/1538
Generating VG image for PPG signal 1261/1538
Generating

Generating VG image for PPG signal 1423/1538
Generating VG image for PPG signal 1424/1538
Generating VG image for PPG signal 1425/1538
Generating VG image for PPG signal 1426/1538
Generating VG image for PPG signal 1427/1538
Generating VG image for PPG signal 1428/1538
Generating VG image for PPG signal 1429/1538
Generating VG image for PPG signal 1430/1538
Generating VG image for PPG signal 1431/1538
Generating VG image for PPG signal 1432/1538
Generating VG image for PPG signal 1433/1538
Generating VG image for PPG signal 1434/1538
Generating VG image for PPG signal 1435/1538
Generating VG image for PPG signal 1436/1538
Generating VG image for PPG signal 1437/1538
Generating VG image for PPG signal 1438/1538
Generating VG image for PPG signal 1439/1538
Generating VG image for PPG signal 1440/1538
Generating VG image for PPG signal 1441/1538
Generating VG image for PPG signal 1442/1538
Generating VG image for PPG signal 1443/1538
Generating VG image for PPG signal 1444/1538
Generating

Generating VG image for PPG signal 70/1273
Generating VG image for PPG signal 71/1273
Generating VG image for PPG signal 72/1273
Generating VG image for PPG signal 73/1273
Generating VG image for PPG signal 74/1273
Generating VG image for PPG signal 75/1273
Generating VG image for PPG signal 76/1273
Generating VG image for PPG signal 77/1273
Generating VG image for PPG signal 78/1273
Generating VG image for PPG signal 79/1273
Generating VG image for PPG signal 80/1273
Generating VG image for PPG signal 81/1273
Generating VG image for PPG signal 82/1273
Generating VG image for PPG signal 83/1273
Generating VG image for PPG signal 84/1273
Generating VG image for PPG signal 85/1273
Generating VG image for PPG signal 86/1273
Generating VG image for PPG signal 87/1273
Generating VG image for PPG signal 88/1273
Generating VG image for PPG signal 89/1273
Generating VG image for PPG signal 90/1273
Generating VG image for PPG signal 91/1273
Generating VG image for PPG signal 92/1273
Generating 

KeyboardInterrupt: 

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch) in enumerate(combined_data_generator):
    print(f"Processing Batch {batch_idx + 1}...")

    # Generate VG images and their inversions
    vg_images = []
    inverted_vg_images = []
    
    for i in range(len(combined_ppg_batch)):
        print(f"Generating VG image for PPG signal {i + 1}/{len(combined_ppg_batch)}")
        
        ppg_signal = combined_ppg_batch[i].flatten()  # Flatten to 1D if necessary
        
        # Generate the VG image for the PPG signal
        vg_image = generate_vg_image(ppg_signal, vg_image_size)
        vg_images.append(vg_image.flatten())
        
        # Generate the inverted VG image
        inverted_vg_image = invert_images([vg_image])[0]
        inverted_vg_images.append(inverted_vg_image.flatten())

    print("Flattening VG images for t-SNE...")
    # Flatten VG images for t-SNE
    #flattened_vg_images = [graph_to_flattened_adjacency_matrix(create_visibility_graph(ppg), vg_image_size) for ppg in combined_ppg_batch]
    flattened_vg_images = vg_images
    print("Combining SBP and DBP values...")
    # Combine SBP and DBP as labels
    y = np.column_stack((combined_seg_sbp_batch.flatten(), combined_seg_dbp_batch.flatten()))  # Assuming they are single values

    print("Applying PCA...")
    # Apply PCA
    pca = PCA(n_components=3)
    X_pca = pca.fit_transform(flattened_vg_images)

    print("Applying t-SNE...")
    # Apply t-SNE
    tsne = TSNE(n_components=3, perplexity=30)
    X_tsne = tsne.fit_transform(X_pca)

    print("Plotting t-SNE results...")
    # Plot t-SNE results
    threshold=80
    plt.figure(figsize=(12, 8))
    plt.scatter(X_tsne[y[:, 0] > threshold, 0], X_tsne[y[:, 0] > threshold, 1], 
            c='red', marker='o', label='High BP')
    plt.scatter(X_tsne[y[:, 0] <= threshold, 0], X_tsne[y[:, 0] <= threshold, 1], 
            c='blue', marker='x', label='Normal BP')
    plt.title('t-SNE Visualization of VG Images for PPG Signals')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.legend()
    plt.show()

    print(f"Batch {batch_idx + 1} processing complete.")

    # Optionally break after the first batch
    break  # Remove this line if you want to process all batches

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

vg_image_dir = 'vg_images'

# Function to load VG images from a given batch index
def load_vg_images(batch_idx):
    vg_images_path = os.path.join(vg_image_dir, f'vg_images_batch_{batch_idx}.npy')
    inverted_vg_images_path = os.path.join(vg_image_dir, f'inverted_vg_images_batch_{batch_idx}.npy')
    
    vg_images = np.load(vg_images_path)
    inverted_vg_images = np.load(inverted_vg_images_path)
    
    return vg_images, inverted_vg_images

# Example: Load VG images from batch 1




for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch) in enumerate(combined_data_generator):
    print(f"Processing Batch {batch_idx + 1}...")

    # Generate VG images and their inversions
    
    vg_images, inverted_vg_images = load_vg_images(batch_idx+1)

    print("Flattening VG images for t-SNE...")
    # Flatten VG images for t-SNE
    #flattened_vg_images = [graph_to_flattened_adjacency_matrix(create_visibility_graph(ppg), vg_image_size) for ppg in combined_ppg_batch]
    flattened_vg_images = vg_images
    print("Combining SBP and DBP values...")
    # Combine SBP and DBP as labels
    y = np.column_stack((combined_seg_sbp_batch.flatten(), combined_seg_dbp_batch.flatten()))  # Assuming they are single values

    print("Applying PCA...")
    # Apply PCA
    pca = PCA(n_components=3)
    X_pca = pca.fit_transform(flattened_vg_images)

    print("Applying t-SNE...")
    # Apply t-SNE
    tsne = TSNE(n_components=3, perplexity=30)
    X_tsne = tsne.fit_transform(X_pca)

    print("Plotting t-SNE results...")
    # Plot t-SNE results
    threshold=80
    plt.figure(figsize=(12, 8))
    plt.scatter(X_tsne[y[:, 0] > threshold, 0], X_tsne[y[:, 0] > threshold, 1], 
            c='red', marker='o', label='High BP')
    plt.scatter(X_tsne[y[:, 0] <= threshold, 0], X_tsne[y[:, 0] <= threshold, 1], 
            c='blue', marker='x', label='Normal BP')
    plt.title('t-SNE Visualization of VG Images for PPG Signals')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.legend()
    plt.show()

    print(f"Batch {batch_idx + 1} processing complete.")

    # Optionally break after the first batch
    break  # Remove this line if you want to process all batches

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=3, perplexity=30)
X_tsne = tsne.fit_transform(X_pca)
threshold=80
plt.figure(figsize=(12, 8))

# Scatter plot for "High BP" (red circles)
plt.scatter(X_tsne[y[:, 0] > threshold, 0], X_tsne[y[:, 0] > threshold, 1], 
            c='red', marker='o', label='High BP')

# Scatter plot for "Normal BP" (blue crosses)
plt.scatter(X_tsne[y[:, 0] <= threshold, 0], X_tsne[y[:, 0] <= threshold, 1], 
            c='blue', marker='x', label='Normal BP')

# Adding title and labels
plt.title('t-SNE Visualization of VG Images for PPG Signals')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')

# Adding legend
plt.legend()

# Show the plot
plt.show()


# PREDICTION FOR ONE BATCH(RIDGE REGRESSION)

In [ ]:
from sklearn.linear_model import Ridge

# Initialize Ridge Regression model
ridge_model = Ridge(alpha=0.1)

ridge_model.fit(X_pca,y)

ridge_model.predict(X_pca)


In [ ]:
print(y)

# RIDGE REGRESSION FOR COMPLETE DATA

In [ ]:
from sklearn.linear_model import Ridge

# Initialize Ridge Regression model
ridge_model = Ridge(alpha=0.1)

# After processing all batches
for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch) in enumerate(combined_data_generator):
    print(f"Processing Batch {batch_idx + 1}...")

    # Generate VG images for the batch
    vg_images_batch = []
    for i in range(len(combined_ppg_batch)):
        ppg_signal = combined_ppg_batch[i].flatten()
        vg_image = generate_vg_image(ppg_signal, vg_image_size)
        vg_images_batch.append(vg_image.flatten())

    # Fit PCA incrementally
    ipca.partial_fit(np.array(vg_images_batch))

    # Transform the current batch
    X_pca_batch = ipca.transform(np.array(vg_images_batch))

    # Prepare the target values (SBP and DBP)
    y_batch = np.column_stack((combined_seg_sbp_batch.flatten(), combined_seg_dbp_batch.flatten()))

    # Incrementally fit Ridge Regression
    ridge_model.partial_fit(X_pca_batch, y_batch)

    print(f"Batch {batch_idx + 1} processing complete.")


In [ ]:
predictions = ridge_model.predict(X_pca_batch)

In [ ]:
for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch, combined_vg_images_batch) in enumerate(combined_data_generator):
    print(f"Batch {batch_idx + 1}:")
    
    # Plot VG images for the first 5 PPG signals in the current batch
    for i in range(len(combined_ppg_batch)):
        ppg_signal = combined_ppg_batch[i].flatten()  # Flatten to 1D if necessary
        
        # Generate the VG image for the PPG signal
        vg_image = generate_vg_image(ppg_signal, vg_image_size)
        
        # Debug: Print PPG signal stats and VG image shape
        print(f"PPG Signal {i + 1} stats - Mean: {np.mean(ppg_signal)}, Std Dev: {np.std(ppg_signal)}")
        print(f"VG Image {i + 1} shape: {vg_image.shape}")
        
        # Plot the PPG signal
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.plot(ppg_signal)
        plt.title(f"PPG Signal {i + 1} (Batch {batch_idx + 1})")
        plt.xlabel("Sample Index")
        plt.ylabel("Amplitude")
        
        # Plot the VG image
        plt.subplot(1, 2, 2)
        plt.imshow(vg_image, cmap='gray', aspect='auto')
        plt.title(f"VG Image {i + 1} (Batch {batch_idx + 1})")
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
    
    # Optionally break after the first batch
    break  # Remove this line if you want to process all batches


In [ ]:
vg_image_size = (224, 224)  # Adjust as necessary

# Generate and plot VG images for the first 5 samples in the first batch
for i in range(min(5, len(combined_ppg_batch))):
    ppg_signal = combined_ppg_batch[i]
    
    # Generate the VG image for the PPG signal
    vg_image = generate_vg_image(ppg_signal, vg_image_size)
    
    # Plot the PPG signal
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(ppg_signal)
    plt.title(f"PPG Signal {i + 1}")
    plt.xlabel("Sample Index")
    plt.ylabel("Amplitude")
    
    # Plot the VG image
    plt.subplot(1, 2, 2)
    plt.imshow(vg_image, cmap='gray', aspect='auto')
    plt.title(f"VG Image {i + 1}")
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
import numpy as np
from tensorflow.keras import applications, models, layers
from tensorflow.keras.preprocessing import image
!pip install --upgrade bottleneck
# Function to preprocess VG images for VGG19
def preprocess_vg_images(vg_images):
    processed_images = []
    for img in vg_images:
        if img.ndim == 2:  # Ensure the image has two dimensions
            img = np.expand_dims(img, axis=-1)  # Add channel dimension
        img = np.repeat(img, 3, axis=-1)  # Convert to 3 channels if single-channel
        image_pil = image.array_to_img(img)
        image_resized = image_pil.resize((224, 224))
        image_array = image.img_to_array(image_resized)
        processed_images.append(image_array)
    processed_images = np.array(processed_images)
    processed_images = applications.vgg19.preprocess_input(processed_images)
    return processed_images


In [ ]:
# Function to create VGG19 feature extractor model
def create_vgg19_feature_extractor(input_shape):
    base_model = applications.VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
    model = models.Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)
    return model


In [ ]:
def ridge_regression(X, y, lambda_reg):
    N, d = X.shape
    X_tilde = np.vstack((X.T, np.ones(N)))
    I = np.eye(d + 1)
    I[-1, -1] = 0
    w_tilde = np.linalg.inv(lambda_reg * I + X_tilde @ X_tilde.T) @ (X_tilde @ y.T)
    return w_tilde

In [ ]:
import os
import numpy as np
import pandas as pd

# Set the path to the folder
folder_path = r'C:\Users\nihal\Desktop\NIHAL_IMP_DOCS\Internship_PPG\processed_npz_files_VitalDB_Final'

# List all files in the folder
files = os.listdir(folder_path)
print(files)

# Filter for NPZ files
npz_files = [f for f in files if f.endswith('.npz')]
print(npz_files)

# Load NPZ files and convert to DataFrames
dataframes = []
for file in npz_files:
    file_path = os.path.join(folder_path, file)
    with np.load(file_path) as npz_file:
        for array_name in npz_file.files:
            array_data = npz_file[array_name]
            print(f"Array name: {array_name}, Shape: {array_data.shape}")
            if array_data.ndim == 0:
                # Ignore scalar (0D) arrays
                print(f"Ignored scalar array: {array_name}")
                continue
            elif array_data.ndim == 1:
                # Convert 1D array to DataFrame with a single column
                df = pd.DataFrame(array_data, columns=[array_name])
            elif array_data.ndim == 2:
                # Directly convert 2D array to DataFrame
                df = pd.DataFrame(array_data)
            elif array_data.ndim == 3:
                # Flatten the 3D array to 2D
                flattened_array = array_data.reshape(array_data.shape[0], -1)
                df = pd.DataFrame(flattened_array)
            else:
                # Ignore arrays with more than 3 dimensions or unknown dimensions
                 print(f"Ignored unsupported array dimension: {array_name} with shape {array_data.shape}")
                continue
                
            dataframes.append(df)

# Optionally, concatenate all dataframes
if dataframes:
    combined_df = pd.concat(dataframes, ignore_index=True)
    print(combined_df.head())
else:
    print("No DataFrames to concatenate.")


['p000001.npz', 'p000003.npz', 'p000005.npz', 'p000006.npz', 'p000008.npz', 'p000010.npz', 'p000012.npz', 'p000013.npz', 'p000016.npz', 'p000019.npz', 'p000022.npz', 'p000023.npz', 'p000024.npz', 'p000027.npz', 'p000028.npz', 'p000030.npz', 'p000033.npz', 'p000035.npz', 'p000037.npz', 'p000038.npz', 'p000041.npz', 'p000043.npz', 'p000047.npz', 'p000050.npz', 'p000051.npz', 'p000052.npz', 'p000053.npz', 'p000056.npz', 'p000058.npz', 'p000059.npz', 'p000061.npz', 'p000063.npz', 'p000064.npz', 'p000066.npz', 'p000068.npz', 'p000071.npz', 'p000073.npz', 'p000074.npz', 'p000079.npz', 'p000085.npz', 'p000088.npz', 'p000089.npz', 'p000093.npz', 'p000097.npz', 'p000101.npz', 'p000102.npz', 'p000103.npz', 'p000107.npz', 'p000109.npz', 'p000110.npz', 'p000111.npz', 'p000113.npz', 'p000116.npz', 'p000119.npz', 'p000120.npz', 'p000121.npz', 'p000122.npz', 'p000124.npz', 'p000128.npz', 'p000129.npz', 'p000130.npz', 'p000131.npz', 'p000134.npz', 'p000135.npz', 'p000136.npz', 'p000137.npz', 'p000139.

In [2]:
!pip install dask

  Obtaining dependency information for dask from https://files.pythonhosted.org/packages/5f/83/1b4cc731fa8550a3041adf1c8e149a318dfee542d6f59a1940552e06df87/dask-2024.7.1-py3-none-any.whl.metadata
  Obtaining dependency information for click>=8.1 from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
  Obtaining dependency information for partd>=1.4.0 from https://files.pythonhosted.org/packages/71/e7/40fb618334dcdf7c5a316c0e7343c5cd82d3d866edc100d98e29bc945ecd/partd-1.4.2-py3-none-any.whl.metadata
  Obtaining dependency information for toolz>=0.10.0 from https://files.pythonhosted.org/packages/b7/8a/d82202c9f89eab30f9fc05380daae87d617e2ad11571ab23d7c13a29bb54/toolz-0.12.1-py3-none-any.whl.metadata
  Obtaining dependency information for importlib-metadata>=4.13.0 from https://files.pythonhosted.org/packages/82/47/bb25ec04985d0693da478797c3d8c1092b140f3a53ccb984fbbd38affa5b/importlib_metadata-8

In [10]:
import os
import numpy as np
import dask.dataframe as dd

def load_npz_to_dask(file_path):
    loaded_data = np.load(file_path)
    dataframes = []
    
    # Handle 'ppg_f' array
    if 'ppg_f' in loaded_data:
        ppg_data = loaded_data['ppg_f']
        if ppg_data.ndim == 3:
            # Convert each 2D slice into a DataFrame
            for i in range(ppg_data.shape[0]):
                ppg_df = dd.from_array(ppg_data[i], columns=[f'ppg_f_{j}' for j in range(ppg_data.shape[2])])
                dataframes.append(ppg_df)
        else:
            print(f"Skipping 'ppg_f' with unexpected shape: {ppg_data.shape}")

    # Handle 'ecg_f' array
    if 'ecg_f' in loaded_data:
        ecg_data = loaded_data['ecg_f']
        if ecg_data.ndim == 3:
            # Convert each 2D slice into a DataFrame
            for i in range(ecg_data.shape[0]):
                ecg_df = dd.from_array(ecg_data[i], columns=[f'ecg_f_{j}' for j in range(ecg_data.shape[2])])
                dataframes.append(ecg_df)
        else:
            print(f"Skipping 'ecg_f' with unexpected shape: {ecg_data.shape}")

    # Handle 'seg_dbp' array
    if 'seg_dbp' in loaded_data:
        seg_dbp_data = loaded_data['seg_dbp']
        if seg_dbp_data.ndim == 2:
            seg_dbp_df = dd.from_array(seg_dbp_data)
            dataframes.append(seg_dbp_df)
        else:
            print(f"Skipping 'seg_dbp' with unexpected shape: {seg_dbp_data.shape}")

    # Handle 'seg_sbp' array
    if 'seg_sbp' in loaded_data:
        seg_sbp_data = loaded_data['seg_sbp']
        if seg_sbp_data.ndim == 2:
            seg_sbp_df = dd.from_array(seg_sbp_data)
            dataframes.append(seg_sbp_df)
        else:
            print(f"Skipping 'seg_sbp' with unexpected shape: {seg_sbp_data.shape}")
    
    return dataframes

def load_data_with_dask(folder_path):
    files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.npz')]
    dfs = [load_npz_to_dask(file_path) for file_path in files]
    dask_dfs = [df for sublist in dfs for df in sublist]
    if dask_dfs:
        combined_dask_df = dd.concat(dask_dfs, axis=1)
        return combined_dask_df

folder_path = r'C:\Users\nihal\Desktop\xyz'
combined_dask_df = load_data_with_dask(folder_path)

# Compute or perform further operations with the Dask DataFrame
if combined_dask_df is not None:
    print(combined_dask_df.head())



KeyboardInterrupt



In [1]:
import os
import shutil
import numpy as np

# Paths
source_dir = r'C:\Users\nihal\Desktop\NIHAL_IMP_DOCS\Internship_PPG\processed_npz_files_VitalDB_Final'
train_dir = 'path_to_train_folder'
val_dir = 'path_to_val_folder'
test_dir = 'path_to_test_folder'

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get list of all npz files
npz_files = [f for f in os.listdir(source_dir) if f.endswith('.npz')]

# Shuffle the list
np.random.shuffle(npz_files)

# Split the files
train_split = int(0.7 * len(npz_files))
val_split = int(0.85 * len(npz_files))

train_files = npz_files[:train_split]
val_files = npz_files[train_split:val_split]
test_files = npz_files[val_split:]

# Move files to their respective directories
for f in train_files:
    shutil.move(os.path.join(source_dir, f), os.path.join(train_dir, f))

for f in val_files:
    shutil.move(os.path.join(source_dir, f), os.path.join(val_dir, f))

for f in test_files:
    shutil.move(os.path.join(source_dir, f), os.path.join(test_dir, f))

print("Files have been successfully split into train, validation, and test folders.")


Files have been successfully split into train, validation, and test folders.


In [12]:
import os
import numpy as np

def check_and_delete_files(folder):
    for file in os.listdir(folder):
        if file.endswith('.npz'):
            file_path = os.path.join(folder, file)
            try:
                # Attempt to load the .npz file
                with np.load(file_path) as data:
                    ppg_shape = data['ppg_f'].shape
                    ecg_shape = data['ecg_f'].shape
                    sbp_shape = data['seg_sbp'].shape
                    dbp_shape = data['seg_dbp'].shape

                # Check if the shapes match the specified shapes
                if ppg_shape == (1250,) and ecg_shape == (1250,) and sbp_shape == () and dbp_shape == ():
                    print(f"Deleting file {file} due to specified shape")
                    try:
                        os.remove(file_path)
                        print(f"File {file} deleted successfully")
                    except Exception as e:
                        print(f"Error deleting file {file}: {e}")
            except Exception as e:
                # Handle any errors that occur
                print(f"Error processing file {file}: {e}")

# Paths
train_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Train_data'
val_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Validation_data'
test_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Test_data'

# Check and delete files with specified shapes

check_and_delete_files(val_dir)
print("jkngnhgfh")
check_and_delete_files(train_dir)
print("jgngfjdfkjjhgddfgjv")
check_and_delete_files(test_dir)

print("Files with the specified shapes have been processed.")


Deleting file p000168.npz due to specified shape
File p000168.npz deleted successfully
Deleting file p000541.npz due to specified shape
File p000541.npz deleted successfully
Deleting file p000554.npz due to specified shape
File p000554.npz deleted successfully
Deleting file p000884.npz due to specified shape
File p000884.npz deleted successfully
Deleting file p000991.npz due to specified shape
File p000991.npz deleted successfully
Deleting file p001049.npz due to specified shape
File p001049.npz deleted successfully
Deleting file p001546.npz due to specified shape
File p001546.npz deleted successfully
Deleting file p001892.npz due to specified shape
File p001892.npz deleted successfully
Deleting file p002362.npz due to specified shape
File p002362.npz deleted successfully
Deleting file p002460.npz due to specified shape
File p002460.npz deleted successfully
Deleting file p002877.npz due to specified shape
File p002877.npz deleted successfully
Deleting file p002923.npz due to specified 

Deleting file p052641.npz due to specified shape
File p052641.npz deleted successfully
Deleting file p053252.npz due to specified shape
File p053252.npz deleted successfully
Deleting file p054110.npz due to specified shape
File p054110.npz deleted successfully
Deleting file p054541.npz due to specified shape
File p054541.npz deleted successfully
Deleting file p057321.npz due to specified shape
File p057321.npz deleted successfully
Deleting file p057568.npz due to specified shape
File p057568.npz deleted successfully
Deleting file p057968.npz due to specified shape
File p057968.npz deleted successfully
Deleting file p058792.npz due to specified shape
File p058792.npz deleted successfully
Deleting file p059845.npz due to specified shape
File p059845.npz deleted successfully
Deleting file p060961.npz due to specified shape
File p060961.npz deleted successfully
Deleting file p062160.npz due to specified shape
File p062160.npz deleted successfully
Deleting file p064101.npz due to specified 

In [5]:
import os
import stat

def remove_readonly(folder):
    for root, dirs, files in os.walk(folder):
        for name in dirs:
            dir_path = os.path.join(root, name)
            try:
                os.chmod(dir_path, stat.S_IWRITE)
            except Exception as e:
                print(f"Error changing permissions for directory {dir_path}: {e}")
        for name in files:
            file_path = os.path.join(root, name)
            try:
                os.chmod(file_path, stat.S_IWRITE)
            except Exception as e:
                print(f"Error changing permissions for file {file_path}: {e}")

# Paths
train_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Train_data'
val_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Val_data'
test_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Test_data'

# Remove read-only attribute
remove_readonly(train_dir)
remove_readonly(val_dir)
remove_readonly(test_dir)

print("Read-only attributes removed.")


Read-only attributes removed.


In [13]:
import numpy as np
import os

def load_npz_files_to_list(folder):
    data_list = []
    for file in os.listdir(folder):
        if file.endswith('.npz'):
            file_path = os.path.join(folder, file)
            try:
                # Load the .npz file and append its content to the list
                with np.load(file_path) as data:
                    data_list.append(data)
            except Exception as e:
                print(f"Error loading file {file_path}: {e}")
    return data_list

# Paths
train_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Train_data'
val_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Validation_data'
test_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Test_data'

# Load data
train_data = load_npz_files_to_list(train_dir)
val_data = load_npz_files_to_list(val_dir)
test_data = load_npz_files_to_list(test_dir)

print(f"Loaded {len(train_data)} training files")
print(f"Loaded {len(val_data)} validation files")
print(f"Loaded {len(test_data)} test files")


Error loading file C:\Users\nihal\Desktop\NIHAL_IMP_DOCS\Internship_PPG\Test_data\p032082.npz: File is not a zip file
Loaded 3623 training files
Loaded 783 validation files
Loaded 780 test files


In [17]:
def inspect_npz_data(data_list):
    for i, data in enumerate(data_list):
        print(f"File {i}:")
        for key in data:
            print(f"  Key: {key}, Shape: {data[key].shape}")

In [19]:
for i, data in enumerate(train_data):
    print(f"Data from file {i}:")
    if 'ppg_f' in data:
        print(f"  ppg_f shape: {data['ppg_f'].shape}")
    if 'ecg_f' in data:
        print(f"  ecg_f shape: {data['ecg_f'].shape}")
    if 'seg_sbp' in data:
        print(f"  seg_sbp shape: {data['seg_sbp'].shape}")
    if 'seg_dbp' in data:
        print(f"  seg_dbp shape: {data['seg_dbp'].shape}")


Data from file 0:


AttributeError: 'NoneType' object has no attribute 'open'

In [21]:
import numpy as np

# Example file path
file_path = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Train_data\\p000012.npz'

# Try to open and inspect one file
try:
    with np.load(file_path) as data:
        for key in data:
            print(f"Key: {key}, Shape: {data[key].shape}")
except Exception as e:
    print(f"Error loading file {file_path}: {e}")


Key: ppg_f, Shape: (1353, 1, 1250)
Key: ecg_f, Shape: (1353, 1, 1250)
Key: seg_dbp, Shape: (1353, 1)
Key: seg_sbp, Shape: (1353, 1)


In [2]:
import numpy as np
import os

def load_npz_files(folder):
    ppg_list = []
    ecg_list = []
    sbp_list = []
    dbp_list = []

    for file in os.listdir(folder):
        if file.endswith('.npz'):
            file_path = os.path.join(folder, file)
            try:
                with np.load(file_path) as data:
                    if 'ppg_f' in data:
                        ppg_list.append(data['ppg_f'])
                    if 'ecg_f' in data:
                        ecg_list.append(data['ecg_f'])
                    if 'seg_sbp' in data:
                        sbp_list.append(data['seg_sbp'])
                    if 'seg_dbp' in data:
                        dbp_list.append(data['seg_dbp'])
            except Exception as e:
                print(f"Error loading file {file_path}: {e}")

    return ppg_list, ecg_list, sbp_list, dbp_list

In [3]:
train_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Train_data'
val_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Validation_data'
test_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Test_data'

train_ppg, train_ecg, train_sbp, train_dbp = load_npz_files(train_dir)
val_ppg, val_ecg, val_sbp, val_dbp = load_npz_files(val_dir)
test_ppg, test_ecg, test_sbp, test_dbp = load_npz_files(test_dir)


print(f"Loaded {len(train_ppg)} training ppg files")
print(f"Loaded {len(train_ecg)} training ecg files")
print(f"Loaded {len(train_sbp)} training sbp files")
print(f"Loaded {len(train_dbp)} training dbp files")

print(f"Loaded {len(val_ppg)} validation ppg files")
print(f"Loaded {len(val_ecg)} validation ecg files")
print(f"Loaded {len(val_sbp)} validation sbp files")
print(f"Loaded {len(val_dbp)} validation dbp files")

print(f"Loaded {len(test_ppg)} test ppg files")
print(f"Loaded {len(test_ecg)} test ecg files")
print(f"Loaded {len(test_sbp)} test sbp files")
print(f"Loaded {len(test_dbp)} test dbp files")

Loaded 783 validation ppg files
Loaded 783 validation ecg files
Loaded 783 validation sbp files
Loaded 783 validation dbp files


In [15]:
val_ppg[2].shape

(9, 1, 1250)

In [ ]:
test_ppg, test_ecg, test_sbp, test_dbp = load_npz_files(test_dir)

In [4]:
import numpy as np
import os
import gc

def process_files_in_batch(file_paths):
    ppg_list = []
    ecg_list = []
    sbp_list = []
    dbp_list = []

    for file_path in file_paths:
        try:
            with np.load(file_path) as data:
                # Append data based on keys
                if 'ppg_f' in data:
                    ppg_list.append(data['ppg_f'])
                if 'ecg_f' in data:
                    ecg_list.append(data['ecg_f'])
                if 'seg_sbp' in data:
                    sbp_list.append(data['seg_sbp'])
                if 'seg_dbp' in data:
                    dbp_list.append(data['seg_dbp'])
        except Exception as e:
            print(f"Error loading file {file_path}: {e}")
    
    return ppg_list, ecg_list, sbp_list, dbp_list

def load_npz_files_in_batches(folder, batch_size=100):
    all_files = [os.path.join(folder, file) for file in os.listdir(folder) if file.endswith('.npz')]
    ppg_list, ecg_list, sbp_list, dbp_list = [], [], [], []

    for i in range(0, len(all_files), batch_size):
        batch_files = all_files[i:i + batch_size]
        ppg, ecg, sbp, dbp = process_files_in_batch(batch_files)
        ppg_list.extend(ppg)
        ecg_list.extend(ecg)
        sbp_list.extend(sbp)
        dbp_list.extend(dbp)
        
        del ppg, ecg, sbp, dbp
        gc.collect()
        
        print(f"Processed batch {i // batch_size + 1} of {len(all_files) // batch_size + 1}")
    
    return ppg_list, ecg_list, sbp_list, dbp_list

In [5]:
# Paths
train_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Train_data'
val_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Validation_data'
test_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Test_data'

In [ ]:
train_ppg, train_ecg, train_sbp, train_dbp = load_npz_files_in_batches(train_dir)

In [6]:
val_ppg, val_ecg, val_sbp, val_dbp = load_npz_files_in_batches(val_dir)

Processed batch 1 of 8
Processed batch 2 of 8
Processed batch 3 of 8
Processed batch 4 of 8
Processed batch 5 of 8
Processed batch 6 of 8
Processed batch 7 of 8
Processed batch 8 of 8


In [ ]:
test_ppg, test_ecg, test_sbp, test_dbp = load_npz_files_in_batches(test_dir)

Processed batch 1 of 8
Processed batch 2 of 8
Processed batch 3 of 8


In [ ]:
print(f"Loaded {len(train_ppg)} training ppg files")
print(f"Loaded {len(train_ecg)} training ecg files")
print(f"Loaded {len(train_sbp)} training sbp files")
print(f"Loaded {len(train_dbp)} training dbp files")

In [ ]:
print(f"Loaded {len(val_ppg)} validation ppg files")
print(f"Loaded {len(val_ecg)} validation ecg files")
print(f"Loaded {len(val_sbp)} validation sbp files")
print(f"Loaded {len(val_dbp)} validation dbp files")

In [ ]:
print(f"Loaded {len(test_ppg)} test ppg files")
print(f"Loaded {len(test_ecg)} test ecg files")
print(f"Loaded {len(test_sbp)} test sbp files")
print(f"Loaded {len(test_dbp)} test dbp files")

# VG GENERATION

In [3]:
import numpy as np
import os

def load_and_validate_data(file_path):
    """
    Load data from a file and return it.
    Return None if data is missing.
    """
    try:
        loaded_data = np.load(file_path)
        
        ppg_f = loaded_data.get('ppg_f')
        ecg_f = loaded_data.get('ecg_f')
        seg_dbp = loaded_data.get('seg_dbp')
        seg_sbp = loaded_data.get('seg_sbp')
        
        if ppg_f is None or ecg_f is None or seg_dbp is None or seg_sbp is None:
            return None

        return ppg_f, ecg_f, seg_dbp, seg_sbp
    
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def combine_data_from_folder(folder_path, batch_size=100):
    """
    Combine data from all valid files in the folder in batches.
    """
    combined_ppg = []
    combined_ecg = []
    combined_seg_dbp = []
    combined_seg_sbp = []

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        if not file_path.endswith('.npz'):
            continue
        
        data = load_and_validate_data(file_path)
        
        if data is None:
            print(f"Skipping invalid file: {file_path}")
            continue
        
        ppg_f, ecg_f, seg_dbp, seg_sbp = data
        
        combined_ppg.append(ppg_f)
        combined_ecg.append(ecg_f)
        combined_seg_dbp.append(seg_dbp)
        combined_seg_sbp.append(seg_sbp)
        
        if len(combined_ppg) >= batch_size:
            combined_ppg = np.concatenate(combined_ppg, axis=0)
            combined_ecg = np.concatenate(combined_ecg, axis=0)
            combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
            combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
            
            yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
            
            combined_ppg = []
            combined_ecg = []
            combined_seg_dbp = []
            combined_seg_sbp = []
            

    if combined_ppg:
        combined_ppg = np.concatenate(combined_ppg, axis=0)
    else:
        combined_ppg = np.array([])
        
    if combined_ecg:
        combined_ecg = np.concatenate(combined_ecg, axis=0)
    else:
        combined_ecg = np.array([])
        
    if combined_seg_dbp:
        combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
    else:
        combined_seg_dbp = np.array([])
        
    if combined_seg_sbp:
        combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
    else:
        combined_seg_sbp = np.array([])

    yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
train_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Train_data'
val_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Validation_data'
test_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Test_data'

In [4]:
train_data_generator = combine_data_from_folder(train_dir, batch_size=100)
val_data_generator = combine_data_from_folder(val_dir, batch_size=100)
test_data_generator = combine_data_from_folder(test_dir, batch_size=100)

In [5]:
import numpy as np
import os
import networkx as nx
from scipy.ndimage import zoom

# Directories for saving VG images
output_dirs = {
    'train': 'vg_images/train',
    'validation': 'vg_images/validation',
    'test': 'vg_images/test'
}

# Ensure the output directories exist
for dir_path in output_dirs.values():
    os.makedirs(dir_path, exist_ok=True)

def create_visibility_graph(ppg_signal):
    n = len(ppg_signal)
    G = nx.Graph()
    G.add_nodes_from(range(n))

    for i in range(n):
        for j in range(i + 1, n):
            visible = True
            for k in range(i + 1, j):
                if ppg_signal[k] >= ppg_signal[i] + (ppg_signal[j] - ppg_signal[i]) * (k - i) / (j - i):
                    visible = False
                    break
            if visible:
                G.add_edge(i, j)
                
    return G

def graph_to_adjacency_matrix_image(G, size):
    adj_matrix = nx.to_numpy_array(G)
    adj_matrix_resized = zoom(adj_matrix, (size[0] / adj_matrix.shape[0], size[1] / adj_matrix.shape[1]), order=0)
    return adj_matrix_resized

def generate_vg_image(ppg_signal, size):
    G = create_visibility_graph(ppg_signal)
    vg_image = graph_to_adjacency_matrix_image(G, size)
    return vg_image

def invert_images(vg_images):
    # Invert pixel values (assuming images are in [0, 255] range)
    inverted_images = [255 - image for image in vg_images]
    return inverted_images

In [9]:
def process_directory(data_generator, output_dir, image_size, save_interval=500):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    batch_number = 1
    vg_images = []
    inverted_vg_images = []

    for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch) in enumerate(data_generator):
        print(f"Processing Batch {batch_idx + 1}...")

        for i in range(len(combined_ppg_batch)):
            print(f"Generating VG image for PPG signal {i + 1}/{len(combined_ppg_batch)}")
            
            ppg_signal = combined_ppg_batch[i].flatten()
            
            # Generate the VG image for the PPG signal
            vg_image = generate_vg_image(ppg_signal, image_size)
            vg_images.append(vg_image.flatten())
            
            # Generate the inverted VG image
            inverted_vg_image = invert_images([vg_image])[0]
            inverted_vg_images.append(inverted_vg_image.flatten())
            
            # Save periodically to avoid high memory usage
            if len(vg_images) >= save_interval:
                np.save(os.path.join(output_dir, f'vg_images_batch_{batch_number}.npy'), vg_images)
                np.save(os.path.join(output_dir, f'inverted_vg_images_batch_{batch_number}.npy'), inverted_vg_images)
                
                # Reset lists and increment batch number
                vg_images = []
                inverted_vg_images = []
                batch_number += 1

        # Save any remaining images after processing the batch
        if vg_images:
            np.save(os.path.join(output_dir, f'vg_images_batch_{batch_number}.npy'), vg_images)
            np.save(os.path.join(output_dir, f'inverted_vg_images_batch_{batch_number}.npy'), inverted_vg_images)

        print(f"Batch {batch_idx + 1} processing complete.")


In [ ]:
process_directory(train_data_generator, output_dirs['train'], image_size=(224, 224))

In [ ]:
process_directory(val_data_generator, output_dirs['validation'], image_size=(224, 224))

Processing Batch 1...
Generating VG image for PPG signal 1/52111
Generating VG image for PPG signal 2/52111
Generating VG image for PPG signal 3/52111
Generating VG image for PPG signal 4/52111
Generating VG image for PPG signal 5/52111
Generating VG image for PPG signal 6/52111
Generating VG image for PPG signal 7/52111
Generating VG image for PPG signal 8/52111
Generating VG image for PPG signal 9/52111
Generating VG image for PPG signal 10/52111
Generating VG image for PPG signal 11/52111
Generating VG image for PPG signal 12/52111
Generating VG image for PPG signal 13/52111
Generating VG image for PPG signal 14/52111
Generating VG image for PPG signal 15/52111
Generating VG image for PPG signal 16/52111
Generating VG image for PPG signal 17/52111
Generating VG image for PPG signal 18/52111
Generating VG image for PPG signal 19/52111
Generating VG image for PPG signal 20/52111
Generating VG image for PPG signal 21/52111
Generating VG image for PPG signal 22/52111
Generating VG image

Generating VG image for PPG signal 185/52111
Generating VG image for PPG signal 186/52111
Generating VG image for PPG signal 187/52111
Generating VG image for PPG signal 188/52111
Generating VG image for PPG signal 189/52111
Generating VG image for PPG signal 190/52111
Generating VG image for PPG signal 191/52111
Generating VG image for PPG signal 192/52111
Generating VG image for PPG signal 193/52111
Generating VG image for PPG signal 194/52111
Generating VG image for PPG signal 195/52111
Generating VG image for PPG signal 196/52111
Generating VG image for PPG signal 197/52111
Generating VG image for PPG signal 198/52111
Generating VG image for PPG signal 199/52111
Generating VG image for PPG signal 200/52111
Generating VG image for PPG signal 201/52111
Generating VG image for PPG signal 202/52111
Generating VG image for PPG signal 203/52111
Generating VG image for PPG signal 204/52111
Generating VG image for PPG signal 205/52111
Generating VG image for PPG signal 206/52111
Generating

Generating VG image for PPG signal 368/52111
Generating VG image for PPG signal 369/52111
Generating VG image for PPG signal 370/52111
Generating VG image for PPG signal 371/52111
Generating VG image for PPG signal 372/52111
Generating VG image for PPG signal 373/52111
Generating VG image for PPG signal 374/52111
Generating VG image for PPG signal 375/52111
Generating VG image for PPG signal 376/52111
Generating VG image for PPG signal 377/52111
Generating VG image for PPG signal 378/52111
Generating VG image for PPG signal 379/52111
Generating VG image for PPG signal 380/52111
Generating VG image for PPG signal 381/52111
Generating VG image for PPG signal 382/52111
Generating VG image for PPG signal 383/52111
Generating VG image for PPG signal 384/52111
Generating VG image for PPG signal 385/52111
Generating VG image for PPG signal 386/52111
Generating VG image for PPG signal 387/52111
Generating VG image for PPG signal 388/52111
Generating VG image for PPG signal 389/52111
Generating

Generating VG image for PPG signal 551/52111
Generating VG image for PPG signal 552/52111
Generating VG image for PPG signal 553/52111
Generating VG image for PPG signal 554/52111
Generating VG image for PPG signal 555/52111
Generating VG image for PPG signal 556/52111
Generating VG image for PPG signal 557/52111
Generating VG image for PPG signal 558/52111
Generating VG image for PPG signal 559/52111
Generating VG image for PPG signal 560/52111
Generating VG image for PPG signal 561/52111
Generating VG image for PPG signal 562/52111
Generating VG image for PPG signal 563/52111
Generating VG image for PPG signal 564/52111
Generating VG image for PPG signal 565/52111
Generating VG image for PPG signal 566/52111
Generating VG image for PPG signal 567/52111
Generating VG image for PPG signal 568/52111
Generating VG image for PPG signal 569/52111
Generating VG image for PPG signal 570/52111
Generating VG image for PPG signal 571/52111
Generating VG image for PPG signal 572/52111
Generating

Generating VG image for PPG signal 734/52111
Generating VG image for PPG signal 735/52111
Generating VG image for PPG signal 736/52111
Generating VG image for PPG signal 737/52111
Generating VG image for PPG signal 738/52111
Generating VG image for PPG signal 739/52111
Generating VG image for PPG signal 740/52111
Generating VG image for PPG signal 741/52111
Generating VG image for PPG signal 742/52111
Generating VG image for PPG signal 743/52111
Generating VG image for PPG signal 744/52111
Generating VG image for PPG signal 745/52111
Generating VG image for PPG signal 746/52111
Generating VG image for PPG signal 747/52111
Generating VG image for PPG signal 748/52111
Generating VG image for PPG signal 749/52111
Generating VG image for PPG signal 750/52111
Generating VG image for PPG signal 751/52111
Generating VG image for PPG signal 752/52111
Generating VG image for PPG signal 753/52111
Generating VG image for PPG signal 754/52111
Generating VG image for PPG signal 755/52111
Generating

...

Generating VG image for PPG signal 1276/52111
Generating VG image for PPG signal 1277/52111
Generating VG image for PPG signal 1278/52111
Generating VG image for PPG signal 1279/52111
Generating VG image for PPG signal 1280/52111
Generating VG image for PPG signal 1281/52111
Generating VG image for PPG signal 1282/52111
Generating VG image for PPG signal 1283/52111
Generating VG image for PPG signal 1284/52111
Generating VG image for PPG signal 1285/52111
Generating VG image for PPG signal 1286/52111
Generating VG image for PPG signal 1287/52111
Generating VG image for PPG signal 1288/52111
Generating VG image for PPG signal 1289/52111
Generating VG image for PPG signal 1290/52111
Generating VG image for PPG signal 1291/52111
Generating VG image for PPG signal 1292/52111
Generating VG image for PPG signal 1293/52111
Generating VG image for PPG signal 1294/52111
Generating VG image for PPG signal 1295/52111
Generating VG image for PPG signal 1296/52111
Generating VG image for PPG signal

Generating VG image for PPG signal 1455/52111
Generating VG image for PPG signal 1456/52111
Generating VG image for PPG signal 1457/52111
Generating VG image for PPG signal 1458/52111
Generating VG image for PPG signal 1459/52111
Generating VG image for PPG signal 1460/52111
Generating VG image for PPG signal 1461/52111
Generating VG image for PPG signal 1462/52111
Generating VG image for PPG signal 1463/52111
Generating VG image for PPG signal 1464/52111
Generating VG image for PPG signal 1465/52111
Generating VG image for PPG signal 1466/52111
Generating VG image for PPG signal 1467/52111
Generating VG image for PPG signal 1468/52111
Generating VG image for PPG signal 1469/52111
Generating VG image for PPG signal 1470/52111
Generating VG image for PPG signal 1471/52111
Generating VG image for PPG signal 1472/52111
Generating VG image for PPG signal 1473/52111
Generating VG image for PPG signal 1474/52111
Generating VG image for PPG signal 1475/52111
Generating VG image for PPG signal

Generating VG image for PPG signal 1634/52111
Generating VG image for PPG signal 1635/52111
Generating VG image for PPG signal 1636/52111
Generating VG image for PPG signal 1637/52111
Generating VG image for PPG signal 1638/52111
Generating VG image for PPG signal 1639/52111
Generating VG image for PPG signal 1640/52111
Generating VG image for PPG signal 1641/52111
Generating VG image for PPG signal 1642/52111
Generating VG image for PPG signal 1643/52111
Generating VG image for PPG signal 1644/52111
Generating VG image for PPG signal 1645/52111
Generating VG image for PPG signal 1646/52111
Generating VG image for PPG signal 1647/52111
Generating VG image for PPG signal 1648/52111
Generating VG image for PPG signal 1649/52111
Generating VG image for PPG signal 1650/52111
Generating VG image for PPG signal 1651/52111
Generating VG image for PPG signal 1652/52111
Generating VG image for PPG signal 1653/52111
Generating VG image for PPG signal 1654/52111
Generating VG image for PPG signal

Generating VG image for PPG signal 1813/52111
Generating VG image for PPG signal 1814/52111
Generating VG image for PPG signal 1815/52111
Generating VG image for PPG signal 1816/52111
Generating VG image for PPG signal 1817/52111
Generating VG image for PPG signal 1818/52111
Generating VG image for PPG signal 1819/52111
Generating VG image for PPG signal 1820/52111
Generating VG image for PPG signal 1821/52111
Generating VG image for PPG signal 1822/52111
Generating VG image for PPG signal 1823/52111
Generating VG image for PPG signal 1824/52111
Generating VG image for PPG signal 1825/52111
Generating VG image for PPG signal 1826/52111
Generating VG image for PPG signal 1827/52111
Generating VG image for PPG signal 1828/52111
Generating VG image for PPG signal 1829/52111
Generating VG image for PPG signal 1830/52111
Generating VG image for PPG signal 1831/52111
Generating VG image for PPG signal 1832/52111
Generating VG image for PPG signal 1833/52111
Generating VG image for PPG signal

Generating VG image for PPG signal 1992/52111
Generating VG image for PPG signal 1993/52111
Generating VG image for PPG signal 1994/52111
Generating VG image for PPG signal 1995/52111
Generating VG image for PPG signal 1996/52111
Generating VG image for PPG signal 1997/52111
Generating VG image for PPG signal 1998/52111
Generating VG image for PPG signal 1999/52111
Generating VG image for PPG signal 2000/52111
Generating VG image for PPG signal 2001/52111
Generating VG image for PPG signal 2002/52111
Generating VG image for PPG signal 2003/52111
Generating VG image for PPG signal 2004/52111
Generating VG image for PPG signal 2005/52111
Generating VG image for PPG signal 2006/52111
Generating VG image for PPG signal 2007/52111
Generating VG image for PPG signal 2008/52111
Generating VG image for PPG signal 2009/52111
Generating VG image for PPG signal 2010/52111
Generating VG image for PPG signal 2011/52111
Generating VG image for PPG signal 2012/52111
Generating VG image for PPG signal

In [4]:
process_directory(test_data_generator, output_dirs['test'], image_size=(224, 224))

In [1]:
import os
import numpy as np
from scipy.signal import butter, filtfilt
import networkx as nx
from scipy.ndimage import zoom
import matplotlib.pyplot as plt

def load_and_validate_data(file_path):
    """
    Load data from a file and validate its presence.
    Return None if data is invalid or shapes are inconsistent.
    """
    try:
        loaded_data = np.load(file_path)
        
        # Extract data
        ppg_f = loaded_data.get('ppg_f')
        ecg_f = loaded_data.get('ecg_f')
        seg_dbp = loaded_data.get('seg_dbp')
        seg_sbp = loaded_data.get('seg_sbp')
        
        # Check if all required data is present and non-null
        if ppg_f is None or ecg_f is None or seg_dbp is None or seg_sbp is None:
            return None
        
        # Check for empty data
        if ppg_f.size == 0 or ecg_f.size == 0 or seg_dbp.size == 0 or seg_sbp.size == 0:
            return None

        return ppg_f, ecg_f, seg_dbp, seg_sbp
    
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    """
    Apply a bandpass filter to the data.
    """
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

def preprocess_signal(signal, fs):
    """
    Preprocess the signal by bandpass filtering and normalization.
    """
    filtered_signal = bandpass_filter(signal, 0.5, 40, fs)
    normalized_signal = (filtered_signal - np.mean(filtered_signal)) / np.std(filtered_signal)
    return normalized_signal

def create_visibility_graph(ppg_signal):
    n = len(ppg_signal)
    G = nx.Graph()
    G.add_nodes_from(range(n))

    for i in range(n):
        for j in range(i + 1, n):
            visible = True
            for k in range(i + 1, j):
                if ppg_signal[k] >= ppg_signal[i] + (ppg_signal[j] - ppg_signal[i]) * (k - i) / (j - i):
                    visible = False
                    break
            if visible:
                G.add_edge(i, j)
                
    return G

def graph_to_adjacency_matrix_image(G, size):
    adj_matrix = nx.to_numpy_array(G)
    adj_matrix_resized = zoom(adj_matrix, (size / adj_matrix.shape[0], size / adj_matrix.shape[1]), order=0)
    return adj_matrix_resized

def graph_to_flattened_adjacency_matrix(G, size):
    adj_matrix = nx.to_numpy_array(G)
    adj_matrix_resized = zoom(adj_matrix, (size / adj_matrix.shape[0], size / adj_matrix.shape[1]), order=0)
    flattened_adj = adj_matrix_resized.flatten()
    return flattened_adj[:size * size]  # Ensure it's the correct size

def generate_vg_image(ppg_signal, size):
    G = create_visibility_graph(ppg_signal)
    vg_image = graph_to_adjacency_matrix_image(G, size)
    return vg_image

def plot_vg_images(vg_images):
    plt.figure(figsize=(10, 10))
    num_images = len(vg_images)
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(vg_images[i], cmap='gray')
        plt.axis('off')
    plt.show()
    
def combine_data_from_folder(folder_path , batch_size=500):
    """
    Combine and preprocess data from all valid files in the folder and generate VG images.
    """
    combined_ppg = []
    combined_ecg = []
    combined_seg_dbp = []
    combined_seg_sbp = []

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        if not file_path.endswith('.npz'):
            continue
        
        data = load_and_validate_data(file_path)
        
        if data is None:
            print(f"Skipping invalid file: {file_path}")
            continue
        
        ppg_f, ecg_f, seg_dbp, seg_sbp = data
        
        if ppg_f.ndim == 3:
            for i in range(ppg_f.shape[0]):
                preprocessed_ppg = preprocess_signal(ppg_f[i], 1000)
                combined_ppg.append(preprocessed_ppg)
        else:
            print(f"Skipping PPG data with unexpected shape: {ppg_f.shape}")
        
        if ecg_f.ndim == 3:
            for i in range(ecg_f.shape[0]):
                preprocessed_ecg = preprocess_signal(ecg_f[i], 1000)
                combined_ecg.append(preprocessed_ecg)
        else:
            print(f"Skipping ECG data with unexpected shape: {ecg_f.shape}")
        
        if seg_dbp.ndim == 2:
            combined_seg_dbp.append(seg_dbp)
        else:
            print(f"Skipping SegDBP data with unexpected shape: {seg_dbp.shape}")
        
        if seg_sbp.ndim == 2:
            combined_seg_sbp.append(seg_sbp)
        else:
            print(f"Skipping SegSBP data with unexpected shape: {seg_sbp.shape}")
        
        if len(combined_ppg) >= batch_size:
            combined_ppg = np.stack(combined_ppg, axis=0)
            combined_ecg = np.stack(combined_ecg, axis=0)
            combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
            combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
            
            yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
            
            combined_ppg = []
            combined_ecg = []
            combined_seg_dbp = []
            combined_seg_sbp = []
            

    if combined_ppg:
        combined_ppg = np.stack(combined_ppg, axis=0)
    else:
        combined_ppg = np.array([])
        
    if combined_ecg:
        combined_ecg = np.stack(combined_ecg, axis=0)
    else:
        combined_ecg = np.array([])
        
    if combined_seg_dbp:
        combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
    else:
        combined_seg_dbp = np.array([])
        
    if combined_seg_sbp:
        combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
    else:
        combined_seg_sbp = np.array([])

    yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
    
def invert_images(vg_images):
    # Invert pixel values (assuming images are in [0, 255] range)
    inverted_images = [255 - image for image in vg_images]
    return inverted_images


folder_path = r'C:\Users\nihal\Desktop\NIHAL_IMP_DOCS\Internship_PPG\Train_data'
vg_image_size = 224  # Define the size for VG images

train_data_generator = combine_data_from_folder(folder_path, 500)

In [2]:
for i, (ppg_batch, ecg_batch, seg_dbp_batch, seg_sbp_batch) in enumerate(train_data_generator):
    print(f"Batch {i + 1}:")
    print(f"  PPG Batch Shape: {ppg_batch.shape}")
    print(f"  ECG Batch Shape: {ecg_batch.shape}")
    print(f"  SegDBP Batch Shape: {seg_dbp_batch.shape}")
    print(f"  SegSBP Batch Shape: {seg_sbp_batch.shape}")
    print()
break

Batch 1:
  PPG Batch Shape: (763, 1, 1250)
  ECG Batch Shape: (763, 1, 1250)
  SegDBP Batch Shape: (763, 1)
  SegSBP Batch Shape: (763, 1)

Batch 2:
  PPG Batch Shape: (1802, 1, 1250)
  ECG Batch Shape: (1802, 1, 1250)
  SegDBP Batch Shape: (1802, 1)
  SegSBP Batch Shape: (1802, 1)

Batch 3:
  PPG Batch Shape: (1273, 1, 1250)
  ECG Batch Shape: (1273, 1, 1250)
  SegDBP Batch Shape: (1273, 1)
  SegSBP Batch Shape: (1273, 1)

Batch 4:
  PPG Batch Shape: (644, 1, 1250)
  ECG Batch Shape: (644, 1, 1250)
  SegDBP Batch Shape: (644, 1)
  SegSBP Batch Shape: (644, 1)

Batch 5:
  PPG Batch Shape: (596, 1, 1250)
  ECG Batch Shape: (596, 1, 1250)
  SegDBP Batch Shape: (596, 1)
  SegSBP Batch Shape: (596, 1)

Batch 6:
  PPG Batch Shape: (822, 1, 1250)
  ECG Batch Shape: (822, 1, 1250)
  SegDBP Batch Shape: (822, 1)
  SegSBP Batch Shape: (822, 1)

Batch 7:
  PPG Batch Shape: (1535, 1, 1250)
  ECG Batch Shape: (1535, 1, 1250)
  SegDBP Batch Shape: (1535, 1)
  SegSBP Batch Shape: (1535, 1)

Batch 8:

KeyboardInterrupt: 

In [12]:
import os
import numpy as np
from scipy.signal import butter, filtfilt
import networkx as nx
from scipy.ndimage import zoom
import matplotlib.pyplot as plt

def load_and_validate_data(file_path):
    """
    Load data from a file and validate its presence.
    Return None if data is invalid or shapes are inconsistent.
    """
    try:
        loaded_data = np.load(file_path)
        
        # Extract data
        ppg_f = loaded_data.get('ppg_f')
        ecg_f = loaded_data.get('ecg_f')
        seg_dbp = loaded_data.get('seg_dbp')
        seg_sbp = loaded_data.get('seg_sbp')
        
        # Check if all required data is present and non-null
        if ppg_f is None or ecg_f is None or seg_dbp is None or seg_sbp is None:
            return None
        
        # Check for empty data
        if ppg_f.size == 0 or ecg_f.size == 0 or seg_dbp.size == 0 or seg_sbp.size == 0:
            return None

        return ppg_f, ecg_f, seg_dbp, seg_sbp
    
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    """
    Apply a bandpass filter to the data.
    """
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

def preprocess_signal(signal, fs):
    """
    Preprocess the signal by bandpass filtering and normalization.
    """
    filtered_signal = bandpass_filter(signal, 0.5, 40, fs)
    normalized_signal = (filtered_signal - np.mean(filtered_signal)) / np.std(filtered_signal)
    return normalized_signal

def create_visibility_graph(ppg_signal):
    n = len(ppg_signal)
    G = nx.Graph()
    G.add_nodes_from(range(n))

    for i in range(n):
        for j in range(i + 1, n):
            visible = True
            for k in range(i + 1, j):
                if ppg_signal[k] >= ppg_signal[i] + (ppg_signal[j] - ppg_signal[i]) * (k - i) / (j - i):
                    visible = False
                    break
            if visible:
                G.add_edge(i, j)
                
    return G

def graph_to_adjacency_matrix_image(G, size):
    adj_matrix = nx.to_numpy_array(G)
    adj_matrix_resized = zoom(adj_matrix, (size / adj_matrix.shape[0], size / adj_matrix.shape[1]), order=0)
    return adj_matrix_resized

def graph_to_flattened_adjacency_matrix(G, size):
    adj_matrix = nx.to_numpy_array(G)
    adj_matrix_resized = zoom(adj_matrix, (size / adj_matrix.shape[0], size / adj_matrix.shape[1]), order=0)
    flattened_adj = adj_matrix_resized.flatten()
    return flattened_adj[:size * size]  # Ensure it's the correct size

def generate_vg_image(ppg_signal, size):
    G = create_visibility_graph(ppg_signal)
    vg_image = graph_to_adjacency_matrix_image(G, size)
    return vg_image

def plot_vg_images(vg_images):
    plt.figure(figsize=(10, 10))
    num_images = len(vg_images)
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(vg_images[i], cmap='gray')
        plt.axis('off')
    plt.show()
    
def combine_data_from_folder(folder_path , batch_size=100):
    """
    Combine and preprocess data from all valid files in the folder and generate VG images.
    """
    combined_ppg = []
    combined_ecg = []
    combined_seg_dbp = []
    combined_seg_sbp = []

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        if not file_path.endswith('.npz'):
            continue
        
        data = load_and_validate_data(file_path)
        
        if data is None:
            print(f"Skipping invalid file: {file_path}")
            continue
        
        ppg_f, ecg_f, seg_dbp, seg_sbp = data
        
        if ppg_f.ndim == 3:
            for i in range(ppg_f.shape[0]):
                preprocessed_ppg = preprocess_signal(ppg_f[i], 1000)
                combined_ppg.append(preprocessed_ppg)
        else:
            print(f"Skipping PPG data with unexpected shape: {ppg_f.shape}")
        
        if ecg_f.ndim == 3:
            for i in range(ecg_f.shape[0]):
                preprocessed_ecg = preprocess_signal(ecg_f[i], 1000)
                combined_ecg.append(preprocessed_ecg)
        else:
            print(f"Skipping ECG data with unexpected shape: {ecg_f.shape}")
        
        if seg_dbp.ndim == 2:
            combined_seg_dbp.append(seg_dbp)
        else:
            print(f"Skipping SegDBP data with unexpected shape: {seg_dbp.shape}")
        
        if seg_sbp.ndim == 2:
            combined_seg_sbp.append(seg_sbp)
        else:
            print(f"Skipping SegSBP data with unexpected shape: {seg_sbp.shape}")
        
        if len(combined_ppg) >= batch_size:
            combined_ppg = np.stack(combined_ppg, axis=0)
            combined_ecg = np.stack(combined_ecg, axis=0)
            combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
            combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
            
            yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
            
            combined_ppg = []
            combined_ecg = []
            combined_seg_dbp = []
            combined_seg_sbp = []
            

    if combined_ppg:
        combined_ppg = np.stack(combined_ppg, axis=0)
    else:
        combined_ppg = np.array([])
        
    if combined_ecg:
        combined_ecg = np.stack(combined_ecg, axis=0)
    else:
        combined_ecg = np.array([])
        
    if combined_seg_dbp:
        combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
    else:
        combined_seg_dbp = np.array([])
        
    if combined_seg_sbp:
        combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
    else:
        combined_seg_sbp = np.array([])

    yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
    
def invert_images(vg_images):
    # Invert pixel values (assuming images are in [0, 255] range)
    inverted_images = [255 - image for image in vg_images]
    return inverted_images


folder_path = r'C:\Users\nihal\Desktop\NIHAL_IMP_DOCS\Internship_PPG\Test_data'
vg_image_size = 224  # Define the size for VG images

test_data_generator = combine_data_from_folder(folder_path, 100)

In [13]:
for i, (ppg_batch, ecg_batch, seg_dbp_batch, seg_sbp_batch) in enumerate(test_data_generator):
    print(f"Batch {i + 1}:")
    print(f"  PPG Batch Shape: {ppg_batch.shape}")
    print(f"  ECG Batch Shape: {ecg_batch.shape}")
    print(f"  SegDBP Batch Shape: {seg_dbp_batch.shape}")
    print(f"  SegSBP Batch Shape: {seg_sbp_batch.shape}")
    print()
break

Batch 1:
  PPG Batch Shape: (916, 1, 1250)
  ECG Batch Shape: (916, 1, 1250)
  SegDBP Batch Shape: (916, 1)
  SegSBP Batch Shape: (916, 1)

Batch 2:
  PPG Batch Shape: (492, 1, 1250)
  ECG Batch Shape: (492, 1, 1250)
  SegDBP Batch Shape: (492, 1)
  SegSBP Batch Shape: (492, 1)

Batch 3:
  PPG Batch Shape: (120, 1, 1250)
  ECG Batch Shape: (120, 1, 1250)
  SegDBP Batch Shape: (120, 1)
  SegSBP Batch Shape: (120, 1)

Batch 4:
  PPG Batch Shape: (1003, 1, 1250)
  ECG Batch Shape: (1003, 1, 1250)
  SegDBP Batch Shape: (1003, 1)
  SegSBP Batch Shape: (1003, 1)

Batch 5:
  PPG Batch Shape: (454, 1, 1250)
  ECG Batch Shape: (454, 1, 1250)
  SegDBP Batch Shape: (454, 1)
  SegSBP Batch Shape: (454, 1)

Batch 6:
  PPG Batch Shape: (135, 1, 1250)
  ECG Batch Shape: (135, 1, 1250)
  SegDBP Batch Shape: (135, 1)
  SegSBP Batch Shape: (135, 1)

Batch 7:
  PPG Batch Shape: (522, 1, 1250)
  ECG Batch Shape: (522, 1, 1250)
  SegDBP Batch Shape: (522, 1)
  SegSBP Batch Shape: (522, 1)

Batch 8:
  PPG B

KeyboardInterrupt: 

In [5]:
import os
import numpy as np
from scipy.signal import butter, filtfilt
import networkx as nx
from scipy.ndimage import zoom
import matplotlib.pyplot as plt

def load_and_validate_data(file_path):
    """
    Load data from a file and validate its presence.
    Return None if data is invalid or shapes are inconsistent.
    """
    try:
        loaded_data = np.load(file_path)
        
        # Extract data
        ppg_f = loaded_data.get('ppg_f')
        ecg_f = loaded_data.get('ecg_f')
        seg_dbp = loaded_data.get('seg_dbp')
        seg_sbp = loaded_data.get('seg_sbp')
        
        # Check if all required data is present and non-null
        if ppg_f is None or ecg_f is None or seg_dbp is None or seg_sbp is None:
            return None
        
        # Check for empty data
        if ppg_f.size == 0 or ecg_f.size == 0 or seg_dbp.size == 0 or seg_sbp.size == 0:
            return None

        return ppg_f, ecg_f, seg_dbp, seg_sbp
    
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    """
    Apply a bandpass filter to the data.
    """
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

def preprocess_signal(signal, fs):
    """
    Preprocess the signal by bandpass filtering and normalization.
    """
    filtered_signal = bandpass_filter(signal, 0.5, 40, fs)
    normalized_signal = (filtered_signal - np.mean(filtered_signal)) / np.std(filtered_signal)
    return normalized_signal

def create_visibility_graph(ppg_signal):
    n = len(ppg_signal)
    G = nx.Graph()
    G.add_nodes_from(range(n))

    for i in range(n):
        for j in range(i + 1, n):
            visible = True
            for k in range(i + 1, j):
                if ppg_signal[k] >= ppg_signal[i] + (ppg_signal[j] - ppg_signal[i]) * (k - i) / (j - i):
                    visible = False
                    break
            if visible:
                G.add_edge(i, j)
                
    return G

def graph_to_adjacency_matrix_image(G, size):
    adj_matrix = nx.to_numpy_array(G)
    adj_matrix_resized = zoom(adj_matrix, (size / adj_matrix.shape[0], size / adj_matrix.shape[1]), order=0)
    return adj_matrix_resized

def graph_to_flattened_adjacency_matrix(G, size):
    adj_matrix = nx.to_numpy_array(G)
    adj_matrix_resized = zoom(adj_matrix, (size / adj_matrix.shape[0], size / adj_matrix.shape[1]), order=0)
    flattened_adj = adj_matrix_resized.flatten()
    return flattened_adj[:size * size]  # Ensure it's the correct size

def generate_vg_image(ppg_signal, size):
    G = create_visibility_graph(ppg_signal)
    vg_image = graph_to_adjacency_matrix_image(G, size)
    return vg_image

def plot_vg_images(vg_images):
    plt.figure(figsize=(10, 10))
    num_images = len(vg_images)
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(vg_images[i], cmap='gray')
        plt.axis('off')
    plt.show()
    
def combine_data_from_folder(folder_path , batch_size=500):
    """
    Combine and preprocess data from all valid files in the folder and generate VG images.
    """
    combined_ppg = []
    combined_ecg = []
    combined_seg_dbp = []
    combined_seg_sbp = []

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        if not file_path.endswith('.npz'):
            continue
        
        data = load_and_validate_data(file_path)
        
        if data is None:
            print(f"Skipping invalid file: {file_path}")
            continue
        
        ppg_f, ecg_f, seg_dbp, seg_sbp = data
        
        if ppg_f.ndim == 3:
            for i in range(ppg_f.shape[0]):
                preprocessed_ppg = preprocess_signal(ppg_f[i], 1000)
                combined_ppg.append(preprocessed_ppg)
        else:
            print(f"Skipping PPG data with unexpected shape: {ppg_f.shape}")
        
        if ecg_f.ndim == 3:
            for i in range(ecg_f.shape[0]):
                preprocessed_ecg = preprocess_signal(ecg_f[i], 1000)
                combined_ecg.append(preprocessed_ecg)
        else:
            print(f"Skipping ECG data with unexpected shape: {ecg_f.shape}")
        
        if seg_dbp.ndim == 2:
            combined_seg_dbp.append(seg_dbp)
        else:
            print(f"Skipping SegDBP data with unexpected shape: {seg_dbp.shape}")
        
        if seg_sbp.ndim == 2:
            combined_seg_sbp.append(seg_sbp)
        else:
            print(f"Skipping SegSBP data with unexpected shape: {seg_sbp.shape}")
        
        if len(combined_ppg) >= batch_size:
            combined_ppg = np.stack(combined_ppg, axis=0)
            combined_ecg = np.stack(combined_ecg, axis=0)
            combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
            combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
            
            yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
            
            combined_ppg = []
            combined_ecg = []
            combined_seg_dbp = []
            combined_seg_sbp = []
            

    if combined_ppg:
        combined_ppg = np.stack(combined_ppg, axis=0)
    else:
        combined_ppg = np.array([])
        
    if combined_ecg:
        combined_ecg = np.stack(combined_ecg, axis=0)
    else:
        combined_ecg = np.array([])
        
    if combined_seg_dbp:
        combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
    else:
        combined_seg_dbp = np.array([])
        
    if combined_seg_sbp:
        combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
    else:
        combined_seg_sbp = np.array([])

    yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
    
def invert_images(vg_images):
    # Invert pixel values (assuming images are in [0, 255] range)
    inverted_images = [255 - image for image in vg_images]
    return inverted_images


folder_path = r'C:\Users\nihal\Desktop\NIHAL_IMP_DOCS\Internship_PPG\Validation_data'
vg_image_size = 224  # Define the size for VG images

val_data_generator = combine_data_from_folder(folder_path, 500)

In [6]:
for i, (ppg_batch, ecg_batch, seg_dbp_batch, seg_sbp_batch) in enumerate(val_data_generator):
    print(f"Batch {i + 1}:")
    print(f"  PPG Batch Shape: {ppg_batch.shape}")
    print(f"  ECG Batch Shape: {ecg_batch.shape}")
    print(f"  SegDBP Batch Shape: {seg_dbp_batch.shape}")
    print(f"  SegSBP Batch Shape: {seg_sbp_batch.shape}")
    print()
break

Batch 1:
  PPG Batch Shape: (956, 1, 1250)
  ECG Batch Shape: (956, 1, 1250)
  SegDBP Batch Shape: (956, 1)
  SegSBP Batch Shape: (956, 1)

Batch 2:
  PPG Batch Shape: (654, 1, 1250)
  ECG Batch Shape: (654, 1, 1250)
  SegDBP Batch Shape: (654, 1)
  SegSBP Batch Shape: (654, 1)

Batch 3:
  PPG Batch Shape: (541, 1, 1250)
  ECG Batch Shape: (541, 1, 1250)
  SegDBP Batch Shape: (541, 1)
  SegSBP Batch Shape: (541, 1)

Batch 4:
  PPG Batch Shape: (1998, 1, 1250)
  ECG Batch Shape: (1998, 1, 1250)
  SegDBP Batch Shape: (1998, 1)
  SegSBP Batch Shape: (1998, 1)

Batch 5:
  PPG Batch Shape: (822, 1, 1250)
  ECG Batch Shape: (822, 1, 1250)
  SegDBP Batch Shape: (822, 1)
  SegSBP Batch Shape: (822, 1)

Batch 6:
  PPG Batch Shape: (1107, 1, 1250)
  ECG Batch Shape: (1107, 1, 1250)
  SegDBP Batch Shape: (1107, 1)
  SegSBP Batch Shape: (1107, 1)

Batch 7:
  PPG Batch Shape: (660, 1, 1250)
  ECG Batch Shape: (660, 1, 1250)
  SegDBP Batch Shape: (660, 1)
  SegSBP Batch Shape: (660, 1)

Batch 8:
  P

KeyboardInterrupt: 